# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names

N = len(data[data.race=='b'])

print('We observe ' + str(sum(data[data.race=='w'].call)) + ' callbacks for white persons, out of '+ str(N))

print('We observe ' + str(sum(data[data.race=='b'].call)) + ' callbacks for black persons, out of '+ str(N))

We observe 235.0 callbacks for white persons, out of 2435
We observe 157.0 callbacks for black persons, out of 2435


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Let us define $p_b$ and $p_w$ as the callback probability for white and black races. The hypothesis we want to test is whether the callback probabilities are independent of the race, i.e.,$H_0 : p_b = p_w$. We are interested in understanding whether $p_w$ is actually larger than $p_b$, i.e., $H_1 : p_w > p_b$.

We aim to perform a one-sided test. Note that the the observations are independent one from the other since the races are assigned randomly. Each observation can be seen as the outcome of a Bernoulli random variable with parameter $p_w$ or $p_b$ depending on the race. 

Note that the number of callback per race $N^{(b)}$ and $N^{(w)}$ follow a binomial distribution with $N = 2435$ and the success probability equal to the race callback probability. 

The CLT applies, and in particular, we have that
$$
\frac{N^{(w)} - N p_w }{\sqrt{N p_w(1 - p_w)} } \xrightarrow{N\rightarrow \infty} \mathcal{N} \big( 0 , 1 \big),
\qquad
\frac{N^{(b)} - N p_b }{\sqrt{N p_b(1 - p_b)} } \xrightarrow{N\rightarrow \infty} \mathcal{N} \big( 0 , 1 \big)
$$

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

def bs_unpaired_statistics (series1, series2, fun, size = 1) :
    ''' Return statistics from permutated bootstrapped samples '''
    series = series1.append(series2)
    stat = np.empty(size)
    for i in range(size):
        temp_perm = np.random.permutation(series)
        stat[i] = fun(temp_perm[:len(series1)]) - fun(temp_perm[len(series1):])
    return stat

diff_obs = w.call.sum() - b.call.sum()
print('We observe ' + str(diff_obs) + ' calls more for white persons. The sampled difference in callback probability is thus ' + str(diff_obs/N))

We observe 78.0 calls more for white persons. The sampled difference in callback probability is thus 0.0320328542094


In [6]:
# Use the bootstrap method to compute the p-value for the hypothesis that the callback is not influenced by the race 
bs_diff = bs_unpaired_statistics (w['call'], b['call'], np.mean, size = 10000)

pi_value = np.sum(bs_diff > diff_obs)/len(bs_diff)
pi_value

print('Bootsrap techinique: The p-value for the test obtained is ' + str(pi_value) + ': the hypothesis can be rejected')

Bootsrap techinique: The p-value for the test obtained is 0.0: the hypothesis can be rejected


### We aim to perform a two-sample t-student for the equality of two means, i.e., $H_0: p_b = p_w$.

Under the hypothesis, the test statistic we use is
$$
 T = \frac{ \bar x_w - \bar x_b }{ \sqrt{\frac{ s_w^2 + s_b^2 }{N}} }
$$
that is distributed as a student random variable with $d$ degrees of freedom, where
$$
 d = \frac{  \big(\frac{s_w^2 + s_b^2}{N}\big)^2 }{ \frac{((s_w^2)^2 + (s_b^2))^2}{N^2(N - 1)}}
   = \frac{  (s_w^2 + s_b^2)^2(N - 1)}  {((s_w^2)^2 + (s_b^2))^2}
$$

In [7]:
var_w = np.var(w['call'], ddof=1)
var_b = np.var(b['call'], ddof=1)

diff_means = np.mean(w['call']) - np.mean(b['call'])

deg = np.floor((((var_w + var_b)**2)*(N-1))/(var_w**2 + var_b**2))

In [8]:
# Look at the confidence interval

from scipy.stats import t 
conf_int_range = t.ppf(0.95, deg)
aux_term = np.sqrt( (var_b + var_w)/N )

conf_interval_l = np.mean(w['call']) - np.mean(b['call']) - conf_int_range*( aux_term )
         
print('\nThe 95% confidence 1-sided interval for the difference in callback probability is [' + str(conf_interval_l)  +', Inf] \n')              

# Look at the p-value for the test for equal means

T = (np.mean(w['call']) - np.mean(b['call']))/ aux_term 

p_value = 1 - t.cdf(T, len(data[data.race=='b']) -1)
print('Frequentist statistic: The statistic takes a value of ' + str(T) + ' and the p-value for the test obtained is ' + str(p_value) + ': the hypothesis can be rejected.\n')


The 95% confidence 1-sided interval for the difference in callback probability is [0.0192253041053, Inf] 

Frequentist statistic: The statistic takes a value of 4.11473837355 and the p-value for the test obtained is 2.00252639442e-05: the hypothesis can be rejected.



### Story
We conclude that racial discrimination continues to be pervasive in cultures throughout the world. The analysis performed showed that the callback probability is not the same for white and black candidates, in particular with 95% of confidence we may claim that such probability is 0.019 higher for white candidates.

That is does not establish that the race is the most important factor in probability callback, but a solution to this discriminatory policy necessary has to be found.

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [40]:
my_dict = {}

for col in data.columns:
    if data[col].dtype == 'float32' :
        corr = np.corrcoef(data['call'], data[col])[0,1]
        if corr != np.nan :
            my_dict[col] = corr
        
    
    if data[col].dtype == 'int8' :
        corr = np.corrcoef(data['call'], data[col])[0,1]
        my_dict[col] = corr    
        if corr != np.nan :
            my_dict[col] = corr
            
    if data[col].dtype == 'int16' :
        corr = np.corrcoef(data['call'], data[col])[0,1]
        my_dict[col] = corr    
        if corr != np.nan :
            my_dict[col] = corr
        

In [41]:
my_dict

{'adid': 0.063178218561005806,
 'bankreal': -0.008995708126726662,
 'branch_emp': nan,
 'branch_sales': nan,
 'busservice': 0.0068819759172010107,
 'call': 1.0,
 'col': -0.0084791840720723991,
 'compreq': -0.024907432828978084,
 'computerskills': -0.028813216729121592,
 'comreq': 0.0024212667779229097,
 'education': -0.0057477955827743956,
 'educreq': -0.033864424187175432,
 'email': 0.025880369289595799,
 'empholes': 0.071888016528191573,
 'eoe': 0.0030921553856934443,
 'expreq': -0.019249538815736301,
 'fed': nan,
 'fracblack': nan,
 'fracblack_empzip': nan,
 'fraccolp': nan,
 'fraccolp_empzip': nan,
 'fracdropout': nan,
 'fracdropout_empzip': nan,
 'fracwhite': nan,
 'fracwhite_empzip': nan,
 'h': 0.025835422657348835,
 'honors': 0.071951068916825525,
 'l': -0.025835422657348835,
 'linc': nan,
 'linc_empzip': nan,
 'lmedhhinc': nan,
 'lmedhhinc_empzip': nan,
 'manager': -0.020268538239853407,
 'manuf': -0.028785400797793445,
 'military': -0.020576686700530807,
 'missind': -0.0075549